In [1]:
import numpy as np
import pandas as pd
from collections import OrderedDict, defaultdict
import glob
import sys 
from icecream import ic
from absl import flags
from sklearn.metrics import roc_auc_score
FLAGS = flags.FLAGS
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import tensorflow as tf
# from tensorflow.python.ops.numpy_ops import np_config
# np_config.enable_numpy_behavior()

from plotly.graph_objs import Scatter,Layout
import plotly
import plotly.graph_objs as go
import plotly.offline as py
import plotly.figure_factory as ff
import plotly.express as px

import gezi
from gezi import tqdm
from wechat import config
from wechat.config import *
from wechat.model import Model
from wechat.dataset import Dataset
ic.disable()

In [2]:
v = 35
model_name = 'doc_encoder'
# model_name = 'doc_encoder2'
model_root = f'../working/offline/{v}'
model_dir = f'{model_root}/{model_name}'
tfrecord_root = '../input/tfrecords'
tfrecord_dir = f'{tfrecord_root}/valid'
ic(model_dir)
ic(tfrecord_dir)
argv = open(f'{model_dir}/command.txt').readline().strip().split()
FLAGS(argv)
# FLAGS.l2_norm = False
config.init()
model = Model()

In [3]:
files=gezi.list_files(f'{tfrecord_dir}/*.tfrec')
dataset = Dataset(files=files)
num_insts = dataset.num_instances
num_insts

get_num_records:   0%|          | 0/96 [00:00<?, ?it/s]

609036

In [4]:
d = dataset.make_batch(10240)
inputs = next(iter(d))[0]

In [5]:
# input_

In [6]:
model(inputs)

<tf.Tensor: shape=(10240, 7), dtype=float32, numpy=
array([[ 1.8251339e-03,  3.7519108e-03,  7.0339092e-04, ...,
         9.0430735e-04,  9.9143363e-04,  3.0927774e-03],
       [ 1.5232568e-03,  3.8839534e-03,  4.6897313e-04, ...,
         6.6606031e-04,  1.0897282e-03,  4.6096118e-03],
       [ 9.2150178e-05,  5.6163892e-03,  1.5509252e-03, ...,
         1.1254626e-05,  1.0555752e-03,  3.5331775e-03],
       ...,
       [ 1.1523033e-03,  3.7744053e-04, -2.2660345e-03, ...,
         1.3519705e-03, -2.8154168e-03,  3.9061210e-03],
       [ 8.7115436e-04,  2.8286930e-03, -1.1081789e-03, ...,
         7.4378191e-04, -4.1503175e-03,  1.6344648e-03],
       [ 2.8446631e-03,  7.2159374e-04, -1.9287819e-04, ...,
        -5.1749608e-04, -2.2117170e-03,  2.2742464e-03]], dtype=float32)>

In [7]:
sum(model.singer_emb.get_weights()[0][2])

0.16530581563711166

In [8]:
sum(model.singer_emb.get_weights()[0][0])

-0.16640743799507618

In [9]:
model.load_weights(f'{model_dir}/model.h5')

In [10]:
# model.summary()

In [11]:
sum(model.singer_emb.get_weights()[0][0])

-0.23709690038231201

In [12]:
sum(model.singer_emb.get_weights()[0][2])

-0.34708589673391543

In [13]:
preds = model(inputs)

In [14]:
inputs['read_comment']

<tf.Tensor: shape=(10240, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [15]:
l = []
for action in ACTIONS:
  l.append(inputs[action])
labels = np.concatenate(l, -1)

In [16]:
# ["read_comment", "like", "click_avatar",  "forward", "comment", "follow", "favorite"]
indexes = []
for i in range(len(inputs['doc'])):
#   if np.sum(labels[i]) > 0:
  if labels[i][1] > 0:
    if len(indexes) < 100:
      if np.max(inputs['read_comments'][i].numpy()) > 0 and np.max(inputs['like'][i].numpy()) > 0:
        print(i, inputs['userid'][i].numpy(), inputs['feedid'][i].numpy(), list(zip(ACTIONS, labels[i], gezi.sigmoid(preds[i]))), inputs['read_comments'][i].numpy())
    indexes.append(i)

690 [182028] [24534] [('read_comment', 0.0, 0.0065632537), ('like', 1.0, 0.012584191), ('click_avatar', 0.0, 0.00067393575), ('forward', 0.0, 5.2600284e-05), ('comment', 0.0, 4.4036783e-06), ('follow', 0.0, 3.0708532e-06), ('favorite', 0.0, 2.1464224e-05)] [3934 1443  344    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
1038 [245355] [48794] [('read_comment', 1.0, 0.14152685), ('like', 1.0, 0.26146147), ('click_avatar', 0.0, 0.0025818408), ('forward', 0.0, 0.00047457463), ('comment', 0.0, 0.00020806472), ('follow', 0.0, 0.0003613309), ('favorite', 0.0, 0.00021312376)] [ 7556  7219 14059 13690  3837 55417 10264   764 13524  3416 32411  3947
 15883 22876 14658 32125  5442 15767  7263 25679 56131 51572 48013 12918
 77359 15720  1890 56551 52599 19064 17263 12711 31102  5608 58844 17189
 14434 28004   1

In [17]:
gezi.sigmoid(preds[0])

array([1.59696105e-03, 4.38864529e-03, 2.62000412e-03, 1.02730955e-04,
       2.53408871e-05, 7.03391997e-05, 3.09527859e-05], dtype=float32)

In [18]:
# model.feats_

In [19]:
model.ori_embs[0]

<tf.Tensor: shape=(31, 128), dtype=float32, numpy=
array([[ 0.25046602,  0.4214798 , -0.01837426, ...,  0.10662767,
         0.24794842,  0.09571373],
       [-0.14641142, -0.43064332, -0.18648393, ..., -0.39898244,
         0.31842244,  0.5444251 ],
       [ 0.08605418,  0.08996293,  0.09055564, ..., -0.03599319,
         0.04719015,  0.02464768],
       ...,
       [-0.05342299, -0.05157678,  0.02096496, ..., -0.08871242,
         0.03009489,  0.02164409],
       [-0.06928851, -0.1845736 , -0.01904272, ..., -0.20728152,
         0.03131673,  0.02348165],
       [-0.0129481 , -0.00327517,  0.01774086, ..., -0.02668421,
        -0.01043929,  0.04652297]], dtype=float32)>

In [20]:
# model.normed_embs[0]

In [21]:
model.x_pooling[0]

<tf.Tensor: shape=(3968,), dtype=float32, numpy=
array([ 0.25046602,  0.4214798 , -0.01837426, ..., -0.02668421,
       -0.01043929,  0.04652297], dtype=float32)>

In [22]:
# model.x_pooling2[0]

In [23]:
len(model.feats)

31

In [24]:
# model.feats_

In [25]:
mats = gezi.get('dot_mat').numpy()

In [26]:
# mats[0]

In [27]:
x_poolings = model.x_pooling.numpy()

In [28]:
feats = np.asarray(list(model.feats_.keys()))
feats

array(['feat_user', 'feat_doc', 'feat_feed', 'feat_device', 'feat_day',
       'feat_author', 'feat_singer', 'feat_song', 'feat_video_time',
       'feat_video_time2', 'feat2_manual_tags', 'feat2_machine_tags',
       'feat2_manual_keys', 'feat2_machine_keys', 'feat2_desc',
       'feat2_ocr', 'feat2_asr', 'encode_cur_doc', 'his_ids_encode_poss',
       'his_ids_encode_read_comments', 'his_ids_encode_comments',
       'his_ids_encode_likes', 'his_ids_encode_click_avatars',
       'his_ids_encode_forwards', 'his_ids_encode_follows',
       'his_ids_encode_favorites', 'his_ids_encode_negs',
       'his_ids_encode_finishs', 'his_ids_encode_unfinishs',
       'his_ids_encode_todays', 'dense_feat'], dtype='<U28')

In [29]:
# x_poolings[0].reshape([-1, FLAGS.emb_dim])

In [30]:
# l = []
# for feat, pooling in zip(feats, x_poolings[0].reshape([-1, FLAGS.emb_dim])):
#   l.append((feat, np.mean(pooling * pooling, -1)))
# feat_weights = l
# sorted(l, key=lambda x: -x[1])

In [31]:
# feat_weights

In [32]:
# mats

In [33]:
feats = list(feats)

In [34]:
def plot(index):
  print(index, 'userid', inputs['userid'][index][0].numpy(), 'feedid', inputs['feedid'][index][0].numpy(), list(zip(ACTIONS, labels[index], gezi.sigmoid(preds[index]))))
  mat = mats[index]
  annotation_text = np.empty(mat.shape)
  for i in range(len(mat)):
    for j in range(len(mat)):
      annotation_text[i][j] = str(round(mat[i][j], 3))
      # if i == j:
      #   mat[i][j] = 0
  # fig = ff.create_annotated_heatmap(z=mat, x=feats, y=feats, annotation_text=annotation_text, colorscale='Viridis')
  fig = ff.create_annotated_heatmap(z=mat, x=feats, y=feats, annotation_text=annotation_text)
  fig.update_layout(title_text=f'<i><b>Cosine Sim of Feats</b></i>',
  # fig.update_layout(title_text=f'<i><b>Dot Sim of Feats</b></i>',
#                     yaxis_autorange='reversed',
                    xaxis={'side': 'bottom'}, 
                   )

  # add colorbar
  fig['data'][0]['showscale'] = True
  fig.show()
  html_file = '../working/debug.html'
  fig.write_html(html_file, auto_open=True)

In [37]:
# plot(690)

In [36]:
abcde

NameError: name 'abcde' is not defined

In [ ]:
plot(0)

In [ ]:
plot(1310)

In [ ]:
mats[0][feats.index('encode_cur_doc')][feats.index('his_ids_encode_read_comments')]

In [ ]:
gezi.metrics.inverse_ratio(input_['read_comment'].numpy().squeeze(-1), mats[:, feats.index('encode_cur_doc'), feats.index('his_ids_encode_read_comments')])

In [ ]:
auc = lambda x, y: 1. - gezi.metrics.inverse_ratio(x, y)
m = {
  'action': [],
  'auc_cosine': [],
  'auc_pred': [],
}
for i, action in enumerate(ACTIONS):
  if action != 'comment' and action != 'follow':
    auc_cosine = auc(input_[action].numpy().squeeze(-1), mats[:, feats.index('encode_cur_doc'), feats.index(f'his_ids_encode_{action}s')])
    auc_pred = auc(input_[action].numpy().squeeze(-1), preds.numpy()[:,i])
    m['action'] = [action]
    m['auc_cosine'] = 